In [1]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('/content/drive/My Drive/IMDB Dataset.csv')

data['review'] = data['review'].str.lower()


In [3]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [4]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result


In [5]:
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

In [6]:
data_without_stopwords.head()

,review,sentiment,review without stopwords,clean_review
0,one of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching just 1 oz epi...,one reviewers mentioned watching just 1 oz epi...
1,a wonderful little production. <br /><br />the...,positive,wonderful little production. <br /><br />the f...,wonderful little production the filming techn...
2,i thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,thought wonderful way spend time hot summer we...
3,basically there's a family where a little boy ...,negative,basically family little boy (jake) thinks zomb...,basically family little boy jake thinks zomb...
4,"petter mattei's ""love in the time of money"" is...",positive,"petter mattei's ""love time money"" visually stu...",petter mattei s love time money visually stu...


In [7]:

reviews = data_without_stopwords['clean_review']
reviews

0        one reviewers mentioned watching just 1 oz epi...
1        wonderful little production  the filming techn...
2        thought wonderful way spend time hot summer we...
3        basically family little boy  jake  thinks zomb...
4        petter mattei s  love time money  visually stu...
                               ...                        
49995    thought movie right good job  wasn t creative ...
49996    bad plot  bad dialogue  bad acting  idiotic di...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    no one expects star trek movies high art  fans...
Name: clean_review, Length: 50000, dtype: object

In [8]:
reviews_list = []
for i in range(len(reviews)):
  reviews_list.append(reviews[i])



In [9]:
sentiment = data_without_stopwords['sentiment']

In [10]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

In [11]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [12]:
X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)

In [13]:
len(Y_train)

40000

In [14]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [15]:
words_to_index = tokenizer.word_index

In [16]:
len(words_to_index)

95419

In [17]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map
    

In [18]:
word_to_vec_map = read_glove_vector('/content/drive/My Drive/glove.6B.50d.txt')

In [19]:
maxLen = 150


In [20]:

vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)






In [21]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [22]:
def conv1d_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = Conv1D(512,3,activation='relu')(embeddings)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  X = Dropout(0.8)(X)
  X = MaxPooling1D(3)(X)

  X = GlobalMaxPooling1D()(X)

  X = Dense(256, activation='relu')(X)
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model
                                    
                      
              
                                      




In [23]:
model = imdb_rating((maxLen,))
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           4770950   
_________________________________________________________________
lstm (LSTM)                  (None, 150, 128)          91648     
_________________________________________________________________
dropout (Dropout)            (None, 150, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 150, 128)          131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 150, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584

In [24]:
model_1d = conv1d_model((maxLen,))
model_1d.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           4770950   
_________________________________________________________________
conv1d (Conv1D)              (None, 148, 512)          77312     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 512)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 47, 256)           393472    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 256)           1968

In [25]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

In [26]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(40000, 150)

In [27]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model_1d.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
model_1d.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 184s 292ms/step - loss: 0.6238 - accuracy: 0.6378
Epoch 2/15
625/625 [==============================] - 183s 293ms/step - loss: 0.4649 - accuracy: 0.7831
Epoch 3/15
625/625 [==============================] - 183s 293ms/step - loss: 0.4196 - accuracy: 0.8063
Epoch 4/15
625/625 [==============================] - 183s 292ms/step - loss: 0.3895 - accuracy: 0.8259
Epoch 5/15
625/625 [==============================] - 182s 291ms/step - loss: 0.3553 - accuracy: 0.8421
Epoch 6/15
625/625 [==============================] - 182s 291ms/step - loss: 0.3354 - accuracy: 0.8553
Epoch 7/15
625/625 [==============================] - 180s 288ms/step - loss: 0.3026 - accuracy: 0.8726
Epoch 8/15
625/625 [==============================] - 181s 290ms/step - loss: 0.2782 - accuracy: 0.8832
Epoch 9/15
625/625 [==============================] - 181s 290ms/step - loss: 0.2438 - accuracy: 0.9006
Epoch 10/15
625/625 [==============================] - 181s 289m

In [ ]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
625/625 [==============================] - 535s 856ms/step - loss: 0.5792 - accuracy: 0.6890
Epoch 2/15
625/625 [==============================] - 536s 858ms/step - loss: 0.4912 - accuracy: 0.7663
Epoch 3/15
625/625 [==============================] - 535s 856ms/step - loss: 0.4715 - accuracy: 0.7778
Epoch 4/15
625/625 [==============================] - 532s 852ms/step - loss: 0.4393 - accuracy: 0.7995
Epoch 5/15
625/625 [==============================] - 534s 855ms/step - loss: 0.4141 - accuracy: 0.8141
Epoch 6/15
625/625 [==============================] - 529s 847ms/step - loss: 0.3913 - accuracy: 0.8253
Epoch 7/15
625/625 [==============================] - 524s 838ms/step - loss: 0.3788 - accuracy: 0.8336
Epoch 8/15
625/625 [==============================] - 520s 832ms/step - loss: 0.3727 - accuracy: 0.8354
Epoch 9/15
625/625 [==============================] - 516s 826ms/step - loss: 0.3643 - accuracy: 0.8416
Epoch 10/15
625/625 [==============================] - 511s 817m

In [29]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [ ]:
model.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 38s 123ms/step - loss: 0.3541 - accuracy: 0.8508


[0.3540983498096466, 0.8507999777793884]

In [30]:
model_1d.evaluate(X_test_indices, Y_test)

313/313 [==============================] - 10s 32ms/step - loss: 0.3657 - accuracy: 0.8352


[0.36570674180984497, 0.8352000117301941]

In [31]:
preds = model_1d.predict(X_test_indices)

In [32]:
n = np.random.randint(0,9999)

X_test[n]


'watched video friend s house  glad not waste money buying one  video cover scene 1975 movie capricorn one  movie starts several clips rocket blow ups  not related manned flight  sibrel s smoking gun short video clip astronauts preparing video broadcast  edits voice over instead letting us listen crew say  video curiously ends showing zapruder film  claims radiation  shielding  star photography  others lead believe extremely ignorant sort ax grind nasa  astronauts  american general  science bad  video '

In [33]:
if preds[n] > 0.5:
  print('predicted sentiment : positive')
else: 
  print('precicted sentiment : negative')

if (Y_test[n] == 1):
  print('correct sentiment : positive')
else:
  print('correct sentiment : negative')


precicted sentiment : negative
correct sentiment : negative


In [34]:
preds[n]

array([0.01440829], dtype=float32)

In [ ]:
Y_test[n]

1

In [ ]:
model_1d.save_weights('/content/drive/My Drive/imdb_weights_con1vd.hdf5')

In [ ]:
reviews_list_idx = tokenizer.texts_to_sequences(reviews_list)

In [ ]:
def add_score_predictions(data, reviews_list_idx):

  data['sentiment score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model.predict(reviews_list_idx)

  data['sentiment score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted sentiment'] = 0

  data['predicted sentiment'] = pred_sentiment

  return data

  

In [ ]:
data = add_score_predictions(data, reviews_list_idx)

In [ ]:
data

,review,sentiment,sentiment score,predicted sentiment
0,one of the other reviewers has mentioned that ...,positive,0.559877,positive
1,a wonderful little production. <br /><br />the...,positive,0.987989,positive
2,i thought this was a wonderful way to spend ti...,positive,0.988815,positive
3,basically there's a family where a little boy ...,negative,0.053478,negative
4,"petter mattei's ""love in the time of money"" is...",positive,0.990116,positive
...,...,...,...,...
49995,i thought this movie did a down right good job...,positive,0.889501,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,0.020020,negative
49997,i am a catholic taught in parochial elementary...,negative,0.296422,negative
49998,i'm going to have to disagree with the previou...,negative,0.707072,positive
